In [1]:
#import libraries
import pandas as pd
import numpy as np

#import the neccessary module
from helper import Model
from helper import Auxiliary

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

set the data


In [2]:
#read the data set, preprocess it, and split it
df_train, df_test = Auxiliary.train_test_split(Auxiliary.preprocess_data(Model.get_csv('BPI_Challenge_2012.csv')))

Start preprocessing data
Finished preprocessing data
Start splitting data
Finished splitting data


In [3]:
#split the data into training and test sets
X_train = df_train.drop(['org:resource', 'case:AMOUNT_REQ', 'Next Event', 'case:REG_DATE', 'time:timestamp'], axis=1)
X_test = df_test.drop(['org:resource', 'case:AMOUNT_REQ', 'Next Event', 'case:REG_DATE', 'time:timestamp'], axis=1)
y_train = df_train['Next Event']
y_test = df_test['Next Event']

In [4]:
#create an instance of the Random Forest model and then fit this to our training data
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

#predict the data
y_pred = rf.predict(X_test)

In [5]:
#compute the accuracy of the predictor
accuracy = accuracy_score(y_test, y_pred)
print(y_pred.size)
print("Accuracy:", accuracy)

17201
Accuracy: 0.7670484274170106
